In [23]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import os
import uuid
import numpy as np
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from transformers import AutoTokenizer, AutoModel
import pinecone



In [7]:
PINECONE_API_KEY ="c5190fed-7028-4d8d-9417-b67ca37c7050"
PINECONE_API_ENV = 'us-east-1'

In [9]:
#Extract data from the PDF
def load_pdf(data):
   loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents

In [10]:
extracted_data = load_pdf("data/")

In [113]:
#extracted_data

In [11]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5039


In [116]:
#text_chunks

In [13]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [119]:
#embeddings

In [15]:
query_result = embeddings.embed_query("hello world")
print("length: ", len(query_result))

length:  384


In [16]:
#query_result

[-0.03447721526026726,
 0.0310231801122427,
 0.006734953261911869,
 0.026108983904123306,
 -0.039362043142318726,
 -0.16030253469944,
 0.06692402064800262,
 -0.006441476754844189,
 -0.04745061323046684,
 0.01475889328867197,
 0.0708753690123558,
 0.055527616292238235,
 0.019193319603800774,
 -0.026251353323459625,
 -0.010109396651387215,
 -0.02694052830338478,
 0.022307440638542175,
 -0.02222662791609764,
 -0.14969268441200256,
 -0.01749303750693798,
 0.0076762959361076355,
 0.05435232073068619,
 0.003254484850913286,
 0.03172596916556358,
 -0.08462141454219818,
 -0.029406029731035233,
 0.05159565061330795,
 0.048124048858881,
 -0.0033147819340229034,
 -0.058279212564229965,
 0.041969284415245056,
 0.02221074514091015,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656217277050018,
 0.06292836368083954,
 -0.03287624940276146,
 -0.09122610837221146,
 -0.03117542900145054,
 0.05269961431622505,
 0.04703483730554581,
 -0.08420301973819733,
 -0.03005622886121273,
 -0.020744772627949715,


In [28]:

# Initialize Pinecone client
pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV )

index_name = 'medical-chatbot'
# Create or get an index
index = pinecone_client.Index(index_name)


# Create the HuggingFaceEmbeddings object
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


# docsearch=PineconeStore.from_texts(
# [t.page_content for t in text_chunks],
# embeddings,
# index_name=index_name
# )

# # Creating Embeddings for Each of the Text Chunks & storing
metadata = {}
for t in text_chunks:
    embedding = embeddings.embed_query(t.page_content)
    metadata["text"] = t.page_content
    index.upsert(vectors=[
        {
            "id": str(uuid.uuid4()),
            "values": embedding,
            "metadata": metadata
        }
    ])



In [29]:

# If we already have an index we can load it like this
docsearch = Pinecone.from_existing_index( index, embeddings)
query = "What are Allergies"

docs= docsearch.similarity_search(query, k=3)

print("Result",docs)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

In [ ]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know. don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [126]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context",  "question"])
chain_type_kwargs={"prompt": PROMPT}

In [125]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docseach
)